# 00 Import & Install Library

In [ ]:
import pandas as pd
import numpy as np
import re
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
from deep_translator import GoogleTranslator

# Stopword Removal & Stemming (Bahasa Indonesia)
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import stanza
from wordcloud import WordCloud

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Text Vectorization
import gensim
from gensim.models import Word2Vec, FastText
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.utils import simple_preprocess

# Modeling
from sklearn.metrics.pairwise import cosine_similarity
from gensim.similarities import WmdSimilarity

# 01 Load Dataset

# 02 Overview & Preprocessing 

# 03 Text Preprocessing

# 04 Text Vectorization

# 05 Similarity Mapping

# 06 Recommendation Modelling

# 07 REST API Development